Let us first,

1. Install the Google agent development tool kit and the pdf extractor packages (pypdf)
2.Import the needful libraries from the respective packages of ADK

In [ ]:
!pip install -U google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.0/290.0 kB 22.5 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 8.5.0
    Uninstalling tenacity-8.5.0:
      Successfully uninstalled tenacity-8.5.0
  Attempting uninstall: google-cloud-storage
    Found existing installation: google-cloud-storage 2.19.0
    Uninstalling google-cloud-storage-2.19.0:
      Successfully uninstalled google-cloud-storage-2.19.0
  Attempting uninstall: google-adk
    Found existing installation: google-adk 1.17.0
    Uninstalling google-adk-1.17.0:
      Successfully uninstalled google-adk-1.17.0


In [ ]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.6/326.6 kB 6.7 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
import os
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY

In [ ]:
from google.genai import types
from google.adk.tools import agent_tool
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.tools import google_search
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search, AgentTool, ToolContext
from google.adk.code_executors import BuiltInCodeExecutor

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


Helper Functions

In [ ]:
def show_python_code_and_result(response):
    for i in range(len(response)):
        # Check if the response contains a valid function call result from the code executor
        if (
            (response[i].content.parts)
            and (response[i].content.parts[0])
            and (response[i].content.parts[0].function_response)
            and (response[i].content.parts[0].function_response.response)
        ):
            response_code = response[i].content.parts[0].function_response.response
            if "result" in response_code and response_code["result"] != "```":
                if "tool_code" in response_code["result"]:
                    print(
                        "Generated Python Code >> ",
                        response_code["result"].replace("tool_code", ""),
                    )
                else:
                    print("Generated Python Response >> ", response_code["result"])


print("✅ Helper functions defined.")

✅ Helper functions defined.


In [ ]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

Extracting the Text contents

In [ ]:
from google.colab import files
from pypdf import PdfReader

uploaded = files.upload()   # choose a PDF
pdf_path = list(uploaded.keys())[0]
print("Loaded:", pdf_path)

def extract_pdf_text(path: str) -> str:
    reader = PdfReader(path)
    pages = [page.extract_text() or "" for page in reader.pages]
    return "\n\n".join(pages)

raw_text = extract_pdf_text(pdf_path)
print("Characters in PDF:", len(raw_text))


Saving ase_1.pdf to ase_1.pdf
Loaded: ase_1.pdf
Characters in PDF: 39082


In [ ]:
document_reader_agent = LlmAgent(
    name="document_reader",
    model=Gemini(model="gemini-2.0-flash", retry_options=retry_config),

    instruction="""
You are a document reader agent.
Take the input document and extract its meaningful content.
Your output MUST be valid Markdown.
Do NOT add your own commentary.
""",
)


In [ ]:
# Context Builder Agent
context_builder_agent = LlmAgent(
    name="context_builder",
    model=Gemini(model="gemini-2.0-flash", retry_options=retry_config),
    instruction="""
    You compress research sections from  into a short, clear context
    (400–600 words) while preserving key technical ideas.
    """
)



In [ ]:
# Summarizer Agent
summarizer_agent = LlmAgent(
    name="summarizer",
    model=Gemini(model="gemini-2.0-flash", retry_options=retry_config),
    instruction="""
    Create a simple, clear explanation of the paper for a student.
    Use headings: Problem, Method, Data, Results, Key Takeaways.
    """,
)

In [ ]:
# Evaluation Agent
evaluation_agent = LlmAgent(
    name="evaluator",
    model=Gemini(model="gemini-2.0-flash", retry_options=retry_config),
    instruction="""
    Evaluate a summary on clarity, correctness, and completeness.
    Give a short bullet-point feedback, but DO NOT rewrite the summary.
    """,
)


In [ ]:
#Tools
document_reader_tool = agent_tool.AgentTool(agent=document_reader_agent)
context_builder_tool = agent_tool.AgentTool(agent=context_builder_agent)
summarizer_tool = agent_tool.AgentTool(agent=summarizer_agent)
evaluation_tool = agent_tool.AgentTool(agent=evaluation_agent)




In [ ]:
#Root Agent
from google.adk.agents import SequentialAgent, LlmAgent

research_analyst_agent = LlmAgent(
    name="analyst",
    model="gemini-2.0-flash",

    instruction="""
You are the RESEARCH ANALYST AGENT.

Your goal is to produce a **detailed, multi-section summary** of the provided research paper.
You MUST use the tools in the following exact sequence:

1. Call `document_reader` and evaluate the content from the document.
2. Call `context_builder` on the document_reader output to compress it into a clear 500–700 word context.
3. Call `summarizer` on the context to generate a student-friendly explanation.
4. Call `evaluator` to validate correctness.

Finally — using the **summarizer output**, produce the final detailed summary.

### Your final summary MUST include these sections:
- **Title**
- **Problem the paper solves**
- **Background / Motivation**
- **Method / Model Explanation**
- **Dataset or Inputs used**
- **Results (numbers if present)**
- **Strengths of the approach**
- **Limitations**
- **Key Takeaways** (5–10 points)
- **Overall Conclusion**

### IMPORTANT RULES
- DO NOT output the raw PDF text.
- DO NOT output intermediate tool results.
- DO NOT output evaluator comments.
- ONLY output the final detailed summary.
- Your final response must be at least **300–600 words**.

Use the tools properly. Do NOT describe the tools. Actually CALL them.
""",
    tools=[document_reader_tool, context_builder_tool, summarizer_tool, evaluation_tool],
)


In [ ]:
# Create session service
session_service = InMemorySessionService()

# Create a runner for any agent using InMemoryRunner
runner = InMemoryRunner(agent=research_analyst_agent)



In [ ]:
response = await runner.run_debug(raw_text)


 ### Created new session: debug_session_id

User > Retrieval-Augmented Generation for
Knowledge-Intensive NLP Tasks
Patrick Lewis†‡ , Ethan Perez?,
Aleksandra Piktus† , Fabio Petroni† , Vladimir Karpukhin† , Naman Goyal† , Heinrich Küttler† ,
Mike Lewis† , Wen-tau Yih† , Tim Rocktäschel†‡ , Sebastian Riedel†‡ , Douwe Kiela†
† Facebook AI Research;‡ University College London;?New York University;
plewis@fb.com
Abstract
Large pre-trained language models have been shown to store factual knowledge
in their parameters, and achieve state-of-the-art results when ﬁne-tuned on down-
stream NLP tasks. However, their ability to access and precisely manipulate
knowledge is still limited, and hence on knowledge-intensive tasks, their perfor-
mance lags behind task-speciﬁc architectures. Additionally, providing provenance
for their decisions and updating their world knowledge remain open research prob-
lems. Pre-trained models with a differentiable access mechanism to explicit non-
parametric memor

analyst > ### Title

Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks

### Problem the paper solves

Large pre-trained language models (PLMs) have limitations in accessing and manipulating knowledge precisely. They require task-specific fine-tuning, which can be resource-intensive. The paper addresses these limitations by introducing a method to incorporate external knowledge into language models to answer specific or up-to-date questions more accurately.

### Background / Motivation

PLMs store significant amounts of information but struggle with tasks needing up-to-date or niche knowledge. Retraining is costly and time-consuming. Retrieval-augmented approaches aim to equip these models with access to external databases, enhancing their ability to address knowledge-intensive tasks without extensive retraining.

### Method / Model Explanation

The Retrieval-Augmented Generation (RAG) model combines a pre-trained sequence-to-sequence model with a non-parametric memory. T